In [1]:
%%bash 
rsync -r LRMfwonLocalhost:/home/debian/test . 

ssh: connect to host localhost port 2211: Connection refused
rsync: connection unexpectedly closed (0 bytes received so far) [Receiver]
rsync error: unexplained error (code 255) at io.c(235) [Receiver=3.1.2]


In [27]:
import sys
sys.path.insert(0, "/home/esr/LRMessanlage/pyLRM")
from pathlib import Path 
import parse
import pandas as pd
import datetime
from bokeh.models import ColumnDataSource,Plot, LinearAxis, Grid
from bokeh.plotting import figure, show, output_notebook
from bokeh.models import DatetimeTickFormatter,Range1d

from bokeh.models.glyphs import Segment

from logging_handler import ax_sensor_log_parser,_ax_sensor_log_row_parser
output_notebook()


def calc_pby(df,time_threshold_s):
    """
    diese funktion splittet eine df von achsdaten in mehrere unterlisten falls 
    die Zeit zwichen Achsdaten Ereignisse > ist als den angegebene time_threshold_s
    """
    pby = []
    pbycnt=0
    ax = []
    axcnt=0
    for toff in df.time_wheel_off.values:
        if toff> time_threshold_s:
            axcnt=0
            pbycnt+=1
        else:
            axcnt+=1
        pby.append(pbycnt)
        ax.append(axcnt)
    return df.assign(passby=pby).assign(ax=ax)

Loading BokehJS ...

### Messung Pfad und name  

In [43]:
path= Path("/home/esr/Documents/Luzern/")
mess_p=Path("/home/esr/Documents/Luzern/Nachmessung2/")
rec_p = path.joinpath("KG")

### Lade achs data
die achs daten sind vom LRM_messung.log abgeholt.


In [44]:
zeitintervalle = [
    (pd.Timestamp("2018-11-13 07:28:00"),pd.Timestamp("2018-11-16 22:00:00")),
                 ]

zeitintervalle = [
    (pd.Timestamp("2018-11-2 "),pd.Timestamp("2018-11-11 22:00:00")),
                 ]

zeitintervalle = [
    (pd.Timestamp("2018-11-19 "),pd.Timestamp("2018-11-26 22:00:00")),
                 ]

#between_times= ['8:00', '12:00']

### Teste assignement

assignement zwischen passby und XLR Daten  

In [45]:
import assign_tools as a
import logging
logger = logging

def filt_rec_passby_list(rp_list,zeitintervalle):
    list_f = []
    for pb in rp_list:
        for (t0,t1) in zeitintervalle:
            if pb['start_rec']>t0 and pb['stop_rec']<t1:
                list_f.append(pb)
                break
    return list_f
    

## Lade passby daten

In [47]:
_,passby_list=a.load_passby_info_from_dir(mess_p,name_match ="*.json")
a.test_passby_duration(mess_p,120)

test_passby_duration: Load passbys from /home/esr/Documents/Luzern/Nachmessung2.
test_passby_duration: Number of passby found: 1613.
test_passby_duration: 0 passby with duration > than 120.


## Filtere liste anhand datenintervalle 

In [48]:
passby_list_f = filt_rec_passby_list(passby_list,zeitintervalle)
print("Remaining passby: {}.".format(len(passby_list_f)))

Remaining passby: 1381.


In [49]:
time_correction=datetime.timedelta(seconds=a.test_passby_time_corrrection(mess_p,9000))

test_passby_time_corrrection: Load passbys from /home/esr/Documents/Luzern/Nachmessung2.
test_passby_time_corrrection: Number of passby found: 1613.
test_passby_time_corrrection: Stats: 
count    1613.000000
mean    -7219.940598
std       378.472773
min     -7236.953222
25%     -7235.834899
50%     -7232.851622
75%     -7231.499481
max      5352.396200
dtype: float64


## Lade XLR rcords 

In [58]:
rec_list = a.load_xl2logs_from_dir(rec_p)


In [59]:
rec_list_f = filt_rec_passby_list(rec_list,[(t0-time_correction,t1-time_correction) for (t0,t1) in zeitintervalle])
print("Remaining passby: {}.".format(len(rec_list_f)))

Remaining passby: 1191.


In [63]:
source_passby = ColumnDataSource(
    pd.DataFrame([{'x0':pb['start_rec'], 'x1':pb['stop_rec'],'y0':1,'y1':1}for pb in passby_list])
)

source_xlr2 = ColumnDataSource(
    pd.DataFrame([{'x0':r['start_rec']+time_correction, 'x1':r['stop_rec']+time_correction,'y0':2,'y1':2}for r in rec_list])
)

plt = figure(x_axis_type="datetime", plot_width=1500, plot_height=100,tools=['xwheel_zoom','xpan'], active_scroll='xwheel_zoom',active_drag='xpan')
glyph1 = Segment(x0="x0", y0="y0", x1="x1", y1="y1", line_color="red", line_width=10)
plt.add_glyph(source_passby, glyph1)
glyph2 = Segment(x0="x0", y0="y0", x1="x1", y1="y1", line_color="blue", line_width=10)
plt.add_glyph(source_xlr2, glyph2)
plt.y_range=Range1d(-0.5,2.5)
show(plt)

## Assign

In [64]:
assigned,not_assigned,not_correctly_assigned=[],[],[]
used_recs =[]

for p in passby_list:
    assigned_recs = a.assign_xl2rec_to_passby(**p, xl2_records=rec_list,logger=logger)
    n_assignements = len(assigned_recs)
    if n_assignements == 0:
        not_assigned.append(p)
        logger.warning("Not assigned: {}, {}.".format(p['path'].parent.parent.name, p['path'].name))
    elif n_assignements == 1:
        p['xl2_rec'] = assigned_recs
        used_recs.append(*assigned_recs)
        assigned.append(p)
    elif n_assignements > 1:
        not_correctly_assigned.append(p)
        logger.warning("Not correctlyassigned: {}, {}.".format(p['path'].parent.parent.name, p['path'].name))
        
logger.info("Correctly assigned {}".format(len(assigned)))
logger.warning("Not assigned {}".format(len(not_assigned)))
logger.warning("Not correctly assigned {}".format(len(not_correctly_assigned)))

<module>       : Not assigned: Luzern, 2018_11_26_08h15m32s_passby.json.
<module>       : Not assigned: Luzern, 2018_11_26_07h33m58s_passby.json.
<module>       : Not assigned: Luzern, 2018_11_25_23h48m21s_passby.json.
<module>       : Not assigned: Luzern, 2018_11_22_23h19m18s_passby.json.
<module>       : Not assigned: Luzern, 2018_11_26_04h49m39s_passby.json.
<module>       : Not assigned: Luzern, 2018_11_22_23h22m24s_passby.json.
<module>       : Not assigned: Luzern, 2018_11_26_08h12m44s_passby.json.
<module>       : Not assigned: Luzern, 2018_11_26_09h05m07s_passby.json.
<module>       : Not assigned: Luzern, 2018_11_24_08h43m26s_passby.json.
<module>       : Not assigned: Luzern, 2018_11_26_06h06m43s_passby.json.
<module>       : Not assigned: Luzern, 2018_11_23_06h12m33s_passby.json.
<module>       : Not assigned: Luzern, 2018_11_22_23h01m46s_passby.json.
<module>       : Not assigned: Luzern, 2018_11_23_16h45m54s_passby.json.
<module>       : Not assigned: Luzern, 2018_11_22_2

In [66]:
source_not_assigned = ColumnDataSource(
    pd.DataFrame([{'x0':pb['start_rec'], 'x1':pb['stop_rec'],'y0':0,'y1':0}for pb in not_assigned])
)

glyph3 = Segment(x0="x0", y0="y0", x1="x1", y1="y1", line_color="orange", line_width=10)
plt.add_glyph(source_not_assigned, glyph2)
show(plt)

In [37]:
len(assigned)

415

In [17]:
used_recs_p = [r['path'] for r in used_recs]

In [18]:
len(set(used_recs_p))

693

In [34]:
len(set(used_recs_p)

693

In [19]:
all_recs_p = set([r['path'] for r in rec_list_f])

In [21]:
[a.load_xl2logs_info(p) for p in all_recs_p.difference(used_recs_p)]

[{'stop_rec': datetime.datetime(2018, 11, 15, 4, 51, 52),
  'start_rec': datetime.datetime(2018, 11, 15, 4, 51, 38),
  'path': PosixPath('/home/esr/Documents/Luzern/XL2KG/2018-11-15_SLM_026_123_Log.txt')},
 {'stop_rec': datetime.datetime(2018, 11, 13, 21, 54, 47),
  'start_rec': datetime.datetime(2018, 11, 13, 21, 54, 34),
  'path': PosixPath('/home/esr/Documents/Luzern/XL2KG/2018-11-13_SLM_067_123_Log.txt')},
 {'stop_rec': datetime.datetime(2018, 11, 14, 20, 52, 4),
  'start_rec': datetime.datetime(2018, 11, 14, 20, 51, 22),
  'path': PosixPath('/home/esr/Documents/Luzern/XL2KG/2018-11-14_SLM_183_123_Log.txt')},
 {'stop_rec': datetime.datetime(2018, 11, 14, 12, 12, 49),
  'start_rec': datetime.datetime(2018, 11, 14, 12, 12, 36),
  'path': PosixPath('/home/esr/Documents/Luzern/XL2KG/2018-11-14_SLM_014_123_Log.txt')},
 {'stop_rec': datetime.datetime(2018, 11, 13, 20, 43, 31),
  'start_rec': datetime.datetime(2018, 11, 13, 20, 43, 18),
  'path': PosixPath('/home/esr/Documents/Luzern/XL2K